In [30]:
import pandas as pd

import json
 
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk, streaming_bulk


import sys
from datetime import datetime

In [ ]:
# https://kibana.kibio.science/app/discover#/?_g=(filters:!(),refreshInterval:(pause:!t,value:0),time:(from:now-15m,to:now))&_a=(columns:!(),filters:!(),index:'6527d100-13e9-11ee-a697-0b0e571db997',interval:auto,query:(language:kuery,query:''),sort:!())


In [3]:
#es_out = Elasticsearch("http://192.168.3.76:9201", basic_auth=('elastic',''),verify_certs=False,ssl_show_warn=False)
print(es_out)

<Elasticsearch(['http://192.168.3.76:9201'])>


In [4]:
from intermine.webservice import Service


In [ ]:
def intermine2es(service_taxon, taxon_id, class_name, field_list, start_pos, end_pos):
    
    service = Service("https://www."+service_taxon+".org/"+service_taxon+"/service")
    #service = Service("https://www.flymine.org/flymine/service")

    # Get a new query on the class (table) you will be querying:
    query = service.new_query(class_name)

    #query.add_view("diseaseId", "diseaseType", "name", "primaryIdentifier")
    for field_name in field_list:
        print(field_name)
        query.add_view(field_name)

    ctr = 0
    res = ''
    jsonStr = ''

    for row in query.rows():
        ctr += 1
        if ctr % 1000 == 0:
            print(ctr, res, jsonStr)

        #print(row["diseaseId"], row["diseaseType"], row["name"], row["primaryIdentifier"], row["alleles.primaryIdentifier"], row["genes.primaryIdentifier"])
        row_dict = row.__dict__
        #print(row_dict)

        if ctr < start_pos:
            continue
        if ctr > end_pos:
            break

        jsonDoc = {}
        jsonDoc['@type'] = 'intermine:'+class_name
        jsonDoc['@context'] = 'http://es.kibio.science/bio2rdf/intermine:'+class_name
        jsonDoc['@context'] = 'http://schema.org/'
        jsonDoc['taxonomy'] = 'taxon:'+taxon_id
        jsonDoc['property'] = 'taxon:7227'

        ctri = 0
        for name in row_dict['views']:
            property = name.split('.')[1]
            property = property.replace('id','intermine_id')
            #print(property, row_dict['data'][ctri])
            jsonDoc[property] = row_dict['data'][ctri]
            ctri += 1

        jsonDoc['@id'] = 'intermine:'+class_name+':' + str(jsonDoc['intermine_id'])
        doc_id = jsonDoc['@id']

        jsonStr = json.dumps(jsonDoc)

        #print(jsonStr)

        #res = es_out.index(index='intermine_'+taxon_id, id=doc_id, document=jsonStr)

#'''
        yield {
            "_index": 'intermine_'+taxon_id,
            "_id": doc_id,
            "_source": jsonStr
            }
#'''
    print(ctr, res)

#intermine2es('flymine','7227','Disease',["name", "identifier", "name", "primaryIdentifier"], 0, 1000000)
#intermine2es('humanmine','9606','Disease',["diseaseId", "diseaseType", "name", "primaryIdentifier"], 0, 1000000)
#intermine2es('humanmine','9606','Gene',["briefDescription", "primaryIdentifier", "cytoLocation", "description","length", "name", "score", "scoreType", "secondaryIdentifier", "symbol"], 0, 1000000)
#intermine2es('flymine','7227','Gene',["briefDescription", "cytoLocation", "primaryIdentifier", "description","length", "name", "score", "scoreType", "secondaryIdentifier", "symbol"]

#bulk(es_out, intermine2es('flymine','7227','Protein',["primaryIdentifier", "ecNumber", "genbankIdentifier", "isFragment","isUniprotCanonical", "length", "md5checksum", "name", "molecularWeight", "primaryAccession", "secondaryIdentifier", "symbol", "uniprotAccession","uniprotName"], 0, 1000000), chunk_size=250, max_retries=20, initial_backoff=5, max_backoff=5)

#bulk(es_out, intermine2es('humanmine','9606','Protein',["isFragment", "isUniprotCanonical", "length", "md5checksum","molecularWeight", "name", "primaryAccession", "primaryIdentifier","secondaryIdentifier", "symbol", "uniprotAccession", "uniprotName"], 0, 1000000), chunk_size=250, max_retries=20, initial_backoff=5, max_backoff=5)

bulk(es_out, intermine2es('humanmine','relation','Relation',["isFragment", "isUniprotCanonical", "length", "md5checksum","molecularWeight", "name", "primaryAccession", "primaryIdentifier","secondaryIdentifier", "symbol", "uniprotAccession", "uniprotName"], 0, 1000000), chunk_size=250, max_retries=20, initial_backoff=5, max_backoff=5)



In [ ]:
query = service.new_query("Gene")

# The view specifies the output columns
query.add_view(
    "briefDescription", "primaryIdentifier", "cytoLocation", "description",
    "length", "name", "score", "scoreType", "secondaryIdentifier", "symbol",
    "diseases.primaryIdentifier", "diseases.name"
)

for row in query.rows():
    print(row["briefDescription"], row["primaryIdentifier"], row["cytoLocation"], \
        row["description"], row["length"], row["name"], row["score"], row["scoreType"], \
        row["secondaryIdentifier"], row["symbol"], row["diseases.primaryIdentifier"], \
        row["diseases.name"])